In [ ]:
# ============================================================
# セル1: Unsloth インストール
# ============================================================
%%capture
!pip install unsloth
# 最新版を取得（Colab用）
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
# ============================================================
# セル2: インポート & GPU確認
# ============================================================
import torch
import json
from pathlib import Path
from datasets import Dataset

print("="*60)
print("🔥 noteAI 2026 World-Class Training")
print("="*60)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name} ({gpu_mem:.1f}GB)")
print("="*60)

In [ ]:
# ============================================================
# セル3: 設定（2026年ベストプラクティス）
# ============================================================

# === モデル選択ガイド（VRAM別推奨 - 2025-2026調査結果） ===
#
# 【あなたのRTX 5060 Ti 16GB VRAM → Qwen3-14B が可能！】
#
# | VRAM     | 推奨モデル            | 4bit VRAM | 学習時VRAM | 備考              |
# |----------|----------------------|-----------|------------|-------------------|
# | ~8GB     | Qwen3-4B-bnb-4bit    | ~4GB      | ~6-8GB     | 120B匹敵の性能    |
# | ~12GB    | Qwen3-14B-bnb-4bit   | ~10GB     | ~12GB      | ★ 12GB可能（実証済）|
# | ~16GB+   | Qwen3-14B-bnb-4bit   | ~10GB     | ~12-14GB   | ★★ 最適！        |
# | ~17.5GB+ | Qwen3-30B-A3B (MoE)  | ~15GB     | ~17.5GB    | 最強クラス        |
#
# 調査結果（2025-2026 Web検索）:
# - YouTube実証: 「Qwen3-14Bを12GB VRAMでファインチューニング可能」
# - Unsloth公式: 「Qwen3-30B-A3Bが17.5GB VRAMで動作」
# - 4bit量子化: 精度低下2-5%、Unsloth最適化で精度低下ゼロ達成
# - rsLoRA + 全層ターゲットで最高精度達成
#
# 量子化の仕組み:
# - bnb = bitsandbytes（量子化ライブラリ）
# - 4bit = 4bit量子化（VRAM 75%削減、速度2.4x向上）
# - nf4 = Normal Float 4（情報密度最適化）

CONFIG = {
    # === モデル ===
    # ★★ Qwen3-14B: RTX 5060 Ti 16GB VRAMで最強モデル！
    # 4bit量子化で約10GB、学習時12-14GB使用（YouTube実証済み）
    "model_name": "unsloth/Qwen3-14B-bnb-4bit",

    # === 代替モデル ===
    # "model_name": "unsloth/Qwen3-8B-bnb-4bit",   # 安全策（~10-12GB使用）
    # "model_name": "unsloth/Qwen3-4B-bnb-4bit",   # 8GB以下のVRAM用

    # === LoRA設定（調査結果に基づく最適値） ===
    "lora_r": 32,              # rank: 16-32で十分（調査結果）
    "lora_alpha": 64,          # 通常 2*r（rsLoRAで自動調整）
    "lora_dropout": 0.0,       # Unslothでは0推奨（最適化済み）
    "use_rslora": True,        # ★ rsLoRA: 高rank安定化（α/√r scaling）
    # DoRAは無効（8xオーバーヘッド - 調査結果）

    # === ターゲットモジュール（全層 - 調査結果で最高精度） ===
    # QLoRA-All: attention + MLP 全層が最高スコア
    "target_modules": [
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention
        "gate_proj", "up_proj", "down_proj",     # MLP/FFN
    ],

    # === トレーニング ===
    "max_seq_length": 512,     # タイトル生成なので短めでOK
    "batch_size": 2,           # ★ 14B用に調整（VRAM節約）
    "gradient_accumulation": 8, # 実効batch=16を維持
    "learning_rate": 2e-4,     # LoRA標準
    "num_epochs": 3,
    "warmup_ratio": 0.03,
    "weight_decay": 0.01,

    # === 出力 ===
    "output_name": "noteai-qwen3-14b-title-v1",
}

print("📋 Training Configuration (2026 Best Practices):")
print("-" * 50)
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

In [ ]:
# ============================================================
# セル4: Google Drive マウント
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# パス設定
DATA_PATH = Path("/content/drive/MyDrive/noteAI/data")
OUTPUT_PATH = Path("/content/drive/MyDrive/noteAI/models")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print(f"📁 Data: {DATA_PATH}")
print(f"📁 Output: {OUTPUT_PATH}")

In [ ]:
# ============================================================
# セル5: Unsloth モデル読み込み（2-5x高速）
# ============================================================
from unsloth import FastLanguageModel

print(f"🔄 Loading: {CONFIG['model_name']}")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=CONFIG["model_name"],
    max_seq_length=CONFIG["max_seq_length"],
    dtype=None,              # 自動検出
    load_in_4bit=True,       # QLoRA
)

print("✅ Model loaded with Unsloth optimization!")

In [ ]:
# ============================================================
# セル6: LoRA設定（rsLoRA + 全層ターゲット）
# ============================================================

model = FastLanguageModel.get_peft_model(
    model,
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],

    # ★ 全層ターゲット（調査結果: QLoRA-All が最高精度）
    target_modules=CONFIG["target_modules"],

    # ★ rsLoRA有効化（高rank安定化）
    use_rslora=CONFIG["use_rslora"],

    # 追加設定
    use_gradient_checkpointing="unsloth",  # Unsloth最適化
    random_state=42,
)

print("\n📊 LoRA Configuration:")
print(f"  Rank (r): {CONFIG['lora_r']}")
print(f"  Alpha: {CONFIG['lora_alpha']}")
print(f"  rsLoRA: {CONFIG['use_rslora']} (α/√r scaling)")
print(f"  Target: {len(CONFIG['target_modules'])} modules (attention + MLP)")
print("\n" + "="*50)
model.print_trainable_parameters()

In [ ]:
# ============================================================
# セル7: データ読み込み
# ============================================================

def load_training_data():
    """トレーニングデータを読み込み"""
    data = []

    # 拡張データ
    augmented_file = DATA_PATH / "augmented" / "augmented_training.jsonl"
    if augmented_file.exists():
        with open(augmented_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    data.append(json.loads(line))
                except:
                    pass
        print(f"  ✓ Augmented: {len(data)} samples")

    # Evol-Instructデータ
    evol_file = DATA_PATH / "processed" / "evol_instruct_data.jsonl"
    evol_count = 0
    if evol_file.exists():
        with open(evol_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    data.append(json.loads(line))
                    evol_count += 1
                except:
                    pass
        print(f"  ✓ Evol-Instruct: {evol_count} samples")

    # 処理済みデータ
    processed_file = DATA_PATH / "processed" / "training_data.jsonl"
    processed_count = 0
    if processed_file.exists():
        with open(processed_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    data.append(json.loads(line))
                    processed_count += 1
                except:
                    pass
        print(f"  ✓ Processed: {processed_count} samples")

    return data

print("📂 Loading training data...")
raw_data = load_training_data()
print(f"\n📊 Total: {len(raw_data)} samples")

In [ ]:
# ============================================================
# セル8: ChatML形式でフォーマット（Qwen3最適）
# ============================================================
from unsloth.chat_templates import get_chat_template

# Qwen3用ChatMLテンプレート
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen3",  # ★ Qwen3用テンプレート
)

def format_to_chatml(entry):
    """
    ChatML形式に変換（Instruct向け最適 - 調査結果）
    """
    # Evol-Instruct形式
    if "instruction" in entry:
        instruction = entry["instruction"]
        input_text = entry.get("input", "")
        output = entry.get("output", "")

        if input_text:
            user_content = f"{instruction}\n\n入力:\n{input_text}"
        else:
            user_content = instruction
        assistant_content = output

    # 通常形式（タイトルデータ）
    else:
        title = entry.get("title", "")
        category = entry.get("category", "一般")

        user_content = f"「{category}」に関する、読者を惹きつけるnote記事のタイトルを1つ生成してください。"
        assistant_content = title

    # ChatML形式
    messages = [
        {"role": "system", "content": "あなたはnote.comの人気記事タイトルを生成する専門AIです。読者の興味を引く、クリックしたくなるタイトルを生成します。"},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": assistant_content},
    ]

    return {"conversations": messages}

# 変換
formatted_data = [format_to_chatml(e) for e in raw_data if e]

print(f"✅ Formatted: {len(formatted_data)} samples")
print("\n📝 Example:")
print("-" * 50)
for msg in formatted_data[0]["conversations"]:
    print(f"[{msg['role']}]: {msg['content'][:100]}..." if len(msg['content']) > 100 else f"[{msg['role']}]: {msg['content']}")

In [ ]:
# ============================================================
# セル9: データセット作成（Completions Only Training）
# ============================================================
from unsloth.chat_templates import standardize_sharegpt, train_on_responses_only
from trl import SFTTrainer
from transformers import TrainingArguments

# データセット作成
dataset = Dataset.from_list(formatted_data)

def apply_chat_template(examples):
    """ChatMLテンプレートを適用"""
    texts = []
    for convs in examples["conversations"]:
        text = tokenizer.apply_chat_template(
            convs,
            tokenize=False,
            add_generation_prompt=False,
        )
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(apply_chat_template, batched=True)

print(f"✅ Dataset ready: {len(dataset)} samples")
print("\n📝 Tokenized example:")
print("-" * 50)
print(dataset[0]["text"][:500])

In [ ]:
# ============================================================
# セル10: トレーニング設定
# ============================================================

training_args = TrainingArguments(
    output_dir=str(OUTPUT_PATH / CONFIG["output_name"]),

    # エポック・バッチ
    num_train_epochs=CONFIG["num_epochs"],
    per_device_train_batch_size=CONFIG["batch_size"],
    gradient_accumulation_steps=CONFIG["gradient_accumulation"],

    # 学習率（LoRA標準）
    learning_rate=CONFIG["learning_rate"],
    warmup_ratio=CONFIG["warmup_ratio"],
    weight_decay=CONFIG["weight_decay"],

    # 最適化
    optim="adamw_8bit",        # Unsloth推奨
    lr_scheduler_type="cosine",

    # 精度
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),

    # ログ
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,

    # その他
    seed=42,
    report_to="none",
)

print("✅ Training arguments configured")

In [ ]:
# ============================================================
# セル11: SFTTrainer + Completions Only Training
# ============================================================

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
    max_seq_length=CONFIG["max_seq_length"],
    dataset_text_field="text",
    packing=False,  # ChatMLではpackingオフ推奨
)

# ★ Completions Only Training（調査結果: +1%精度向上）
# アシスタントの回答部分のみで学習
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n",
)

print("✅ Trainer ready with Completions-Only Training")
print("   (Training only on assistant responses for better accuracy)")

In [ ]:
# ============================================================
# セル12: トレーニング実行
# ============================================================
import time

print("="*60)
print("🚀 TRAINING START")
print("="*60)
print(f"Model: {CONFIG['model_name']}")
print(f"Samples: {len(dataset)}")
print(f"Epochs: {CONFIG['num_epochs']}")
print(f"Effective batch: {CONFIG['batch_size'] * CONFIG['gradient_accumulation']}")
print("="*60)

start_time = time.time()

# トレーニング
trainer_stats = trainer.train()

elapsed = time.time() - start_time
print("\n" + "="*60)
print("✅ TRAINING COMPLETE")
print("="*60)
print(f"Time: {elapsed/60:.1f} minutes")
print(f"Loss: {trainer_stats.training_loss:.4f}")

In [ ]:
# ============================================================
# セル13: モデル保存（複数形式）
# ============================================================

save_path = OUTPUT_PATH / CONFIG["output_name"]

# 1. LoRAアダプターのみ保存
print("💾 Saving LoRA adapter...")
model.save_pretrained(str(save_path / "lora_adapter"))
tokenizer.save_pretrained(str(save_path / "lora_adapter"))
print(f"   → {save_path / 'lora_adapter'}")

# 2. マージ済み16bit（推論用）
print("\n💾 Saving merged 16bit model...")
model.save_pretrained_merged(
    str(save_path / "merged_16bit"),
    tokenizer,
    save_method="merged_16bit",
)
print(f"   → {save_path / 'merged_16bit'}")

# 3. GGUF形式（llama.cpp / Ollama用）
print("\n💾 Saving GGUF (Q4_K_M)...")
model.save_pretrained_gguf(
    str(save_path / "gguf"),
    tokenizer,
    quantization_method="q4_k_m",
)
print(f"   → {save_path / 'gguf'}")

print("\n" + "="*60)
print("✅ ALL MODELS SAVED")
print("="*60)

In [ ]:
# ============================================================
# セル14: 推論テスト
# ============================================================
from unsloth.chat_templates import get_chat_template

# 推論モード
FastLanguageModel.for_inference(model)

def generate_title(category: str, temperature: float = 0.7):
    """タイトル生成"""
    messages = [
        {"role": "system", "content": "あなたはnote.comの人気記事タイトルを生成する専門AIです。読者の興味を引く、クリックしたくなるタイトルを生成します。"},
        {"role": "user", "content": f"「{category}」に関する、読者を惹きつけるnote記事のタイトルを1つ生成してください。"},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=64,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
        )

    # 生成部分のみ抽出
    generated = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    return generated.strip().split("\n")[0]

# テスト
test_categories = [
    "副業",
    "英語学習",
    "プログラミング",
    "投資",
    "ブログ運営",
    "AI活用",
    "転職",
    "ライティング",
]

print("\n" + "="*60)
print("🎯 GENERATION TEST")
print("="*60)

for cat in test_categories:
    title = generate_title(cat)
    print(f"\n【{cat}】")
    print(f"  → {title}")

In [ ]:
# ============================================================
# セル15: 複数タイトル生成（バリエーション）
# ============================================================

def generate_multiple_titles(category: str, n: int = 5):
    """複数のタイトルバリエーションを生成"""
    titles = []
    for i in range(n):
        # 温度を変えてバリエーション
        temp = 0.6 + (i * 0.1)
        title = generate_title(category, temperature=temp)
        titles.append(title)
    return titles

# テスト
print("\n" + "="*60)
print("🎨 TITLE VARIATIONS TEST")
print("="*60)

for cat in ["副業", "AI活用"]:
    print(f"\n【{cat}】のタイトル候補:")
    print("-" * 40)
    titles = generate_multiple_titles(cat, n=5)
    for i, title in enumerate(titles, 1):
        print(f"  {i}. {title}")

## ✅ トレーニング完了

### 保存先
- **LoRAアダプター**: `/content/drive/MyDrive/noteAI/models/{output_name}/lora_adapter/`
- **マージ済み16bit**: `/content/drive/MyDrive/noteAI/models/{output_name}/merged_16bit/`
- **GGUF (Q4_K_M)**: `/content/drive/MyDrive/noteAI/models/{output_name}/gguf/`

### 使用方法

#### Unslothで推論
```python
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained("path/to/lora_adapter")
```

#### Ollamaで使用
```bash
ollama create noteai -f Modelfile
ollama run noteai "副業に関するタイトルを生成"
```

### 技術仕様（2026年世界最高水準）
| 項目 | 設定 | 根拠 |
|------|------|------|
| Framework | Unsloth | 2-5x高速、80%省VRAM |
| Base Model | Qwen2.5-7B-Instruct | 日本語最強クラス |
| Method | QLoRA + rsLoRA | 高rank安定化 |
| Target | All Layers | QLoRA-All最高精度 |
| Training | Completions Only | +1%精度向上 |
| Format | ChatML | Instruct向け最適 |